    Here's a list of six sites attached. The location is good for all except Erkuta but I think I figured it out based on the proximity to the research station over there and a big bend in the river.

In [36]:
import geopandas as gpd
import shapely.geometry
import pyproj
import os
import wget

### Read data, create geometry of points

In [26]:
points = gpd.read_file("./data/gas_emission_crater_locations.csv")
points["Long (E)"] = points["Long (E)"].astype(float)
points["Lat (N)"] = points["Lat (N)"].astype(float)
points["geometry"] = points.apply(lambda point: shapely.geometry.Point(point["Long (E)"], point["Lat (N)"]), axis=1)
points

,Site Name,Lat (N),Long (E),Formation timing,Notes,geometry
0,Yamal GEC-1,69.970965,68.369575,Between June 2013 and June 2014,,POINT (68.36957 69.97097)
1,Yamal GEC-2,70.144600,68.495900,Before 02 May 2013,,POINT (68.49590 70.14460)
2,Yamal GEC-3,70.062400,69.100200,Unknown,,POINT (69.10020 70.06240)
3,AntGEC,69.794600,75.035000,Before late October 2013,,POINT (75.03500 69.79460)
4,Erkuta GEC,68.110328,69.551738,Before 01 June 2017,My best guess at the location based off a figu...,POINT (69.55174 68.11033)
5,SeYkh GEC,70.302100,71.745800,Unknown,,POINT (71.74580 70.30210)


### Project points to EPSG 3413

In [27]:
wgs84 = pyproj.CRS("EPSG:4326")
sea_ice_polar_stereographic_north = pyproj.CRS("EPSG:3413")
projection = pyproj.Transformer.from_crs(wgs84, sea_ice_polar_stereographic_north, always_xy=True).transform

points["geometry"] = points["geometry"].apply(lambda point: shapely.ops.transform(projection, point))
points

,Site Name,Lat (N),Long (E),Formation timing,Notes,geometry
0,Yamal GEC-1,69.970965,68.369575,Between June 2013 and June 2014,,POINT (2011415.459 869149.377)
1,Yamal GEC-2,70.144600,68.495900,Before 02 May 2013,,POINT (1991732.324 865860.396)
2,Yamal GEC-3,70.062400,69.100200,Unknown,,POINT (1990857.945 890562.136)
3,AntGEC,69.794600,75.035000,Before late October 2013,,POINT (1913976.195 1106594.131)
4,Erkuta GEC,68.110328,69.551738,Before 01 June 2017,My best guess at the location based off a figu...,POINT (2182419.400 996966.815)
5,SeYkh GEC,70.302100,71.745800,Unknown,,POINT (1923761.814 969478.940)


### Load Strip file

In [29]:
strips = gpd.read_file("../data/ArcticDEM_Strip_Index_Rel7/ArcticDEM_Strip_Index_Rel7.shp")

### Download Strip data for each site

In [37]:
for _, point in points.iterrows():
    print(f'Downloading all strips for {point["Site Name"]}')

    save_dir = os.path.join("data", point["Site Name"])
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    valid_strips = strips[strips["geometry"].apply(lambda strip: strip.contains(point["geometry"]))]
    for _, strip in valid_strips.iterrows():
        wget.download(strip["fileurl"], save_dir)
    

In [34]:
for _, point in points.iterrows():
    print(f'Downloading all strips for {point["Site Name"]}')

    save_dir = os.path.join("data", point["Site Name"])
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    valid_strips = strips[strips["geometry"].apply(lambda strip: strip.contains(point["geometry"]))]
    for _, strip in valid_strips.iterrows():
        wget.download(strip["fileurl"], save_dir)
    

,name,pairname,catalogid1,catalogid2,nd_value,resolution,algm_ver,creationda,raster,fileurl,...,num_gcps,meanresz,active,qc,rel_ver,acquisit_1,sensor2,st_area_sh,st_length_,geometry
0,SETSM_GE01_20090818_1050410001E0CF00_105041000...,GE01_20090818_1050410001E0CF00_1050410001D80200,1050410001E0CF00,1050410001D80200,-9999.0,2.0,SETSM 3.2.8,2017-12-15,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,...,8,-0.146,1,2,7,2009-08-18,GE01,6.040148e+08,101983.612304,"POLYGON ((-319964.000 -2761546.000, -319804.00..."
1,SETSM_GE01_20090818_1050410001DB4300_105041000...,GE01_20090818_1050410001DB4300_1050410001D8F000,1050410001DB4300,1050410001D8F000,-9999.0,2.0,SETSM 3.1.2,2017-11-04,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,...,0,0.000,1,0,7,2009-08-18,GE01,2.298394e+08,58049.966245,"POLYGON ((542616.000 -2323500.000, 542952.000 ..."
2,SETSM_GE01_20090816_10504100038FD200_105041000...,GE01_20090816_10504100038FD200_1050410002BAFF00,10504100038FD200,1050410002BAFF00,-9999.0,2.0,SETSM 3.1.2,2017-11-04,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,...,0,0.000,1,0,7,2009-08-16,GE01,2.618302e+08,62877.693904,"POLYGON ((542284.000 -2323402.000, 543268.000 ..."
3,SETSM_GE01_20170412_1050010009368D00_105001000...,GE01_20170412_1050010009368D00_1050010009368B00,1050010009368D00,1050010009368B00,-9999.0,2.0,SETSM 3.2.5,2017-09-22,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,...,134,-0.426,1,0,7,2017-04-12,GE01,2.553323e+08,62640.055387,"POLYGON ((1111890.000 -2618082.000, 1112034.00..."
4,SETSM_GE01_20170608_105001000A33BA00_105001000...,GE01_20170608_105001000A33BA00_105001000A33BC00,105001000A33BA00,105001000A33BC00,-9999.0,2.0,SETSM 3.2.8,2018-04-07,None,http://data.pgc.umn.edu/elev/dem/setsm/ArcticD...,...,15,0.008,1,0,7,2017-06-08,GE01,2.826451e+08,65288.949158,"POLYGON ((1110886.000 -2617508.000, 1110894.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206527,SETSM_WV02_20180410_103001007C238F00_103001007...,WV02_20180410_103001007C238F00_103001007D4B1A00,103001007C238F00,103001007D4B1A00,-9999.0,2.0,SETSM 3.4.0,2019-04-12,None,/mnt/pgc/data/elev/dem/setsm/ArcticDEM/geocell...,...,0,0.000,0,0,v3.0,2018-04-10,WV02,0.000000e+00,0.000000,"POLYGON ((-345214.000 -2679472.000, -345182.00..."
206528,SETSM_WV03_20180723_104001003F49B700_104001004...,WV03_20180723_104001003F49B700_1040010040C0F000,104001003F49B700,1040010040C0F000,-9999.0,2.0,SETSM 3.3.2,2019-04-12,None,/mnt/pgc/data/elev/dem/setsm/ArcticDEM/geocell...,...,0,0.000,0,0,v3.0,2018-07-23,WV03,0.000000e+00,0.000000,"POLYGON ((-21030.000 -3188674.000, -21006.000 ..."
206529,SETSM_WV02_20180830_103001008576C100_103001008...,WV02_20180830_103001008576C100_10300100835D1900,103001008576C100,10300100835D1900,-9999.0,2.0,SETSM 3.4.0,2019-04-12,None,/mnt/pgc/data/elev/dem/setsm/ArcticDEM/geocell...,...,0,0.000,0,0,v3.0,2018-08-30,WV02,0.000000e+00,0.000000,"POLYGON ((-307752.000 -1854710.000, -305598.00..."
206530,SETSM_WV03_20180426_104001003C1CFF00_104001003...,WV03_20180426_104001003C1CFF00_104001003C0A9A00,104001003C1CFF00,104001003C0A9A00,-9999.0,2.0,SETSM 3.3.2,2019-04-12,None,/mnt/pgc/data/elev/dem/setsm/ArcticDEM/geocell...,...,0,0.000,0,0,v3.0,2018-04-26,WV03,0.000000e+00,0.000000,"POLYGON ((-116186.000 -3230792.000, -115644.00..."


### List number of available strips for each site

In [39]:
for _, point in points.iterrows():
    valid_strips = strips[strips["geometry"].apply(lambda strip: strip.contains(point["geometry"]))]
    print(point["Site Name"], "-", len(valid_strips))

Yamal GEC-1 - 19
Yamal GEC-2 - 9
Yamal GEC-3 - 7
AntGEC - 9
Erkuta GEC - 3
SeYkh GEC - 6
